# 📓 Tutorial 4: Exporting Synthetic Grids to Pandapower & OpenDSS

This notebook is the final step in the `bayesgrid` workflow. It shows how to take the synthetic `.csv` files generated by `bayesgrid` and convert them into simulation-ready network files for both `pandapower` and `OpenDSS`.

This process allows for powerful stochastic simulations, as you can generate hundreds of network variations and run analyses on each one.

## Imports

First, let's import all the necessary libraries. We'll be using our new `save_synthetic_network` function.

In [1]:
import os
import pandapower as pp
import pandapower.networks as pn
import numpy as np
import pandas as pd
import warnings

# Import our custom Bayesian model classes
from bayesgrid import (
    BayesianPowerModel, 
    BayesianFrequencyModel, 
    BayesianDurationModel, 
    BayesianImpedanceModel
)

# Import our new EXPORTER function
from bayesgrid import save_synthetic_network

# Suppress warnings for a cleaner tutorial
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

print("Libraries imported successfully.")

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install gxx`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.


Helper functions for saving are defined.
Libraries imported successfully.






## 🌎 Step 1: Define Your Inputs

This tutorial is the final step in the generation pipeline. We assume you have already completed **Tutorial 1 (Pandapower)** or **Tutorial 2 (OSM)**.

As a result, you should have a folder (e.g., `new_pp_synthetic_net`) containing the four CSV files with all your synthetic samples:

  * `bus_power_and_phase_SAMPLES.csv`
  * `bus_frequency_SAMPLES.csv`
  * `bus_duration_SAMPLES.csv`
  * `line_impedance_SAMPLES.csv`

To export a network, you need two things:

1.  **The Base Network:** The original `pandapower` network you used as a template (e.g., `case118` or your OSM network). This provides the topology (buses, lines, and lengths).
2.  **Paths to the Synthetic Data:** The file paths to the four CSV files listed above.


In [7]:
# 1. Load the Base Network
# We'll use case118, but this could be any 'net' object
print("Loading base network topology (case118)...")
net = pn.case118()

# CRITICAL: We must ensure bus and line names are set to their index
# This is how the exporter matches data from the CSVs
net.bus['name'] = net.bus.index
net.line['name'] = net.line.index

# 2. Define data paths
# This is the folder created in the 'pandapower' or 'osm' tutorial
SYNTHETIC_DATA_FOLDER = 'new_pp_synthetic_net' 

# Define the full paths to each required CSV file
path_power = os.path.join(SYNTHETIC_DATA_FOLDER, 'bus_power_and_phase_SAMPLES.csv')
path_freq = os.path.join(SYNTHETIC_DATA_FOLDER, 'bus_frequency_SAMPLES.csv')
path_dur = os.path.join(SYNTHETIC_DATA_FOLDER, 'bus_duration_SAMPLES.csv')
path_imp = os.path.join(SYNTHETIC_DATA_FOLDER, 'line_impedance_SAMPLES.csv')

# 3. Define output folders
PANDAPOWER_EXPORT_FOLDER = 'synthetic_net_pandapower'
OPENDSS_EXPORT_FOLDER = 'synthetic_net_opendss'

print(f"Base network loaded. Synthetic data will be read from '{SYNTHETIC_DATA_FOLDER}'")

Loading base network topology (case118)...
Base network loaded. Synthetic data will be read from 'new_pp_synthetic_net'


# Step 2: Exporting in pandapower format (json)

In [8]:
save_synthetic_network(
    base_net=net,
    path_power_phase=path_power,
    path_frequency=path_freq,
    path_duration=path_dur,
    path_impedance=path_imp,
    output_path=PANDAPOWER_EXPORT_FOLDER,
    format='pandapower'
)

Loading synthetic data from specified paths...
Found 100 unique synthetic samples.
Exporting 100 Pandapower networks to: synthetic_net_pandapower
  ... generating pandapower sample 99 of 99


Pandapower export complete. 100 files created.


## Verify Pandapower Export

Let's check the folder we just created. You should see a `.json` file for every sample. We'll load `net_sample_0.json` to inspect it.

In [9]:
print(f"Checking output folder: '{PANDAPOWER_EXPORT_FOLDER}'")
files = os.listdir(PANDAPOWER_EXPORT_FOLDER)

print(f"Found {len(files)} files.")
print("--- First 5 files ---")
for f in files[:5]:
    print(f)

# --- Load one sample to verify ---
print("\n--- Loading 'net_sample_0.json' to verify contents ---")
sample_net_path = os.path.join(PANDAPOWER_EXPORT_FOLDER, 'net_sample_0.json')

try:
    net_syn = pp.from_json(sample_net_path)

    print("\n--- Synthetic Bus Data (Sample 0) ---")
    bus_cols = ['name', 'phase', 'P_A', 'P_B', 'P_C', 'CAIFI_FIC', 'CAIDI_DIC']
    print(net_syn.bus[bus_cols].head())

    print("\n--- Synthetic Line Data (Sample 0) ---")
    line_cols = ['name', 'R1_ohm_per_km', 'X1_ohm_per_km']
    print(net_syn.line[line_cols].head())

except FileNotFoundError:
    print(f"Error: Could not find '{sample_net_path}'. Did the export in Step 2 run correctly?")

Checking output folder: 'synthetic_net_pandapower'
Found 100 files.
--- First 5 files ---
net_sample_0.json
net_sample_1.json
net_sample_10.json
net_sample_11.json
net_sample_12.json

--- Loading 'net_sample_0.json' to verify contents ---

--- Synthetic Bus Data (Sample 0) ---
   name phase  P_A  P_B          P_C  CAIFI_FIC  CAIDI_DIC
0     0     C  0.0  0.0  2880.050019          1   5.555562
1     1     C  0.0  0.0  2139.256917          2   4.685801
2     2     C  0.0  0.0  2508.449974          0   4.646143
3     3     C  0.0  0.0  1790.500666          1   3.659418
4     4     C  0.0  0.0  2393.523167          1   6.477275

--- Synthetic Line Data (Sample 0) ---
   name  R1_ohm_per_km  X1_ohm_per_km
0     0       0.342819       0.399866
1     1       0.645988       0.524995
2     2       0.698201       0.245689
3     3       0.435270       0.365472
4     4       0.667691       0.297652


# Step 3: Export to OpenDSS

This export does the same thing: it will read all samples and generate one complete, runnable `.dss` file *for each sample*.

**Note:** OpenDSS does **not** have a standard field for reliability data. This exporter will *only* export the load (Power/Phase) and line (Impedance) data. The reliability data (`CAIFI_FIC`, `CAIDI_DIC`) will be ignored.

In [10]:
save_synthetic_network(
    base_net=net,
    path_power_phase=path_power,
    path_frequency=path_freq,
    path_duration=path_dur,
    path_impedance=path_imp,
    output_path=OPENDSS_EXPORT_FOLDER,
    format='opendss'
)

Loading synthetic data from specified paths...
Found 100 unique synthetic samples.
Exporting 100 OpenDSS networks to: synthetic_net_opendss
  ... generating opendss sample 99 of 99


OpenDSS export complete. 100 files created.


## Verify OpenDSS Export

Let's check the folder we just created. You should see a `.dss` file for every sample.


In [12]:
print(f"Checking output folder: '{OPENDSS_EXPORT_FOLDER}'")

try:
    files = os.listdir(OPENDSS_EXPORT_FOLDER)
    print(f"Found {len(files)} files.")
    print("--- First 5 files ---")
    for f in files[:5]:
        print(f)

    print("\n--- Contents of 'master_sample_0.dss' (first 20 lines) ---")
    with open(os.path.join(OPENDSS_EXPORT_FOLDER, 'master_sample_0.dss'), 'r') as f:
        for i in range(20):
            print(f.readline().strip())
            
except FileNotFoundError:
    print(f"Error: Could not find folder '{OPENDSS_EXPORT_FOLDER}'. Did the export in Step 3 run correctly?")

Checking output folder: 'synthetic_net_opendss'
Found 100 files.
--- First 5 files ---
master_sample_0.dss
master_sample_1.dss
master_sample_10.dss
master_sample_11.dss
master_sample_12.dss

--- Contents of 'master_sample_0.dss' (first 20 lines) ---
clear
new circuit.synthetic_net basekv=157.6864406779661 pu=1.0 phases=3
edit Vsource.source bus1=bus_68.1.2.3

!---------------- LINES ----------------

new linecode.LCode_0 nphases=1 rmatrix=[0.401979] xmatrix=[1.396524] units=km
new line.line_0 bus1=bus_0.3 bus2=bus_1.3 linecode=LCode_0 length=1.0 units=km
new linecode.LCode_1 nphases=1 rmatrix=[0.705148] xmatrix=[1.521668] units=km
new line.line_1 bus1=bus_0.3 bus2=bus_2.3 linecode=LCode_1 length=1.0 units=km
new linecode.LCode_2 nphases=1 rmatrix=[0.757361] xmatrix=[1.242348] units=km
new line.line_2 bus1=bus_3.3 bus2=bus_4.3 linecode=LCode_2 length=1.0 units=km
new linecode.LCode_3 nphases=1 rmatrix=[0.494430] xmatrix=[1.362130] units=km
new line.line_3 bus1=bus_2.3 bus2=bus_4.3 linec



## ✅ Tutorial Complete!

You have successfully exported your synthetic data to both `pandapower` and `OpenDSS` formats.

  * You have a **`pandapower` folder** (`.json` files) containing one full network for each sample, including all synthetic data.
  * You have an **OpenDSS folder** (`.dss` files) containing one full, runnable simulation file for each sample.